# 04 - Comparaison de scénarios & Monte Carlo

Ce notebook compare plusieurs scénarios électoraux et quantifie l'incertitude via simulation Monte Carlo.

## Contenu

1. Comparaison côte à côte de scénarios
2. Analyse de sensibilité
3. Simulation Monte Carlo (N=10 000)
4. Probabilités de victoire

In [ ]:
import sys
sys.path.insert(0, '..')

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display

from paris_elections.scenarios.scenario import (
    Scenario,
    ScenarioComparator,
    scenario_gauche_unie,
    scenario_droite_unie,
    scenario_fragmentation,
)
from paris_elections.scenarios.montecarlo import run_monte_carlo
from paris_elections.viz.charts import bar_seats_comparison, histogram_mc
from paris_elections.config import MAYOR_ABSOLUTE_MAJORITY, POLITICAL_FAMILIES

## 1. Chargement des scénarios

In [ ]:
# Scénarios prédéfinis
scenarios = [
    scenario_gauche_unie(),
    scenario_droite_unie(),
    scenario_fragmentation(),
]

# Scénario personnalisé : statu quo (inspiré 2020)
statu_quo = Scenario(
    name="Statu quo",
    description="Projection basée sur les rapports de force 2020",
    paris_scores={
        "PS": 22.0,
        "LFI": 10.0,
        "EELV": 8.0,
        "PCF": 3.0,
        "REN": 15.0,
        "LR": 18.0,
        "RN": 8.0,
        "REC": 5.0,
        "DIV": 11.0,
    },
    participation=0.45,
)
scenarios.append(statu_quo)

for sc in scenarios:
    print(f"• {sc.name}: {sc.description}")

## 2. Comparaison des résultats

In [ ]:
# Simuler tous les scénarios
comparator = ScenarioComparator()
for sc in scenarios:
    comparator.add(sc)

results = comparator.run_all()

# Tableau des sièges
seats_table = comparator.seats_table()
seats_df = pd.DataFrame(seats_table).T
seats_df.index.name = 'Scénario'
display(seats_df)

In [ ]:
# Graphique barres empilées
fig = bar_seats_comparison(seats_table)
fig.show()

In [ ]:
# Résumé par coalition
coalition_summary = comparator.coalition_summary()
coal_df = pd.DataFrame(coalition_summary).T
display(coal_df)

## 3. Analyse de sensibilité

Comment les résultats varient-ils si on modifie légèrement les scores ?

In [ ]:
# Variantes du scénario "Statu quo"
base = statu_quo
variants = []

# Variation de la participation
for part in [0.35, 0.40, 0.45, 0.50, 0.55]:
    v = base.variant(name=f"Participation {int(part*100)}%", participation=part)
    variants.append(v)

# Simuler
sensitivity_results = {}
for v in variants:
    res = v.simulate()
    # Sièges de la liste en tête
    top = max(res.total_seats_conseil, key=res.total_seats_conseil.get)
    sensitivity_results[v.name] = {
        'top_liste': top,
        'top_seats': res.total_seats_conseil[top],
        'majorite': res.total_seats_conseil[top] >= MAYOR_ABSOLUTE_MAJORITY,
    }

sens_df = pd.DataFrame(sensitivity_results).T
display(sens_df)

## 4. Simulation Monte Carlo

Quantification de l'incertitude avec N=10 000 itérations.

In [ ]:
# Monte Carlo sur le scénario "Statu quo"
# Réduire N pour la démo (10 000 prend du temps)
N = 1000  # Augmenter à 10_000 pour des résultats robustes

print(f"Simulation Monte Carlo avec N={N} itérations...")
mc_result = run_monte_carlo(statu_quo, n_iterations=N, seed=42)
print("Terminé !")

In [ ]:
# Résumé des résultats
summary = mc_result.summary_table()
summary_df = pd.DataFrame(summary).T
summary_df = summary_df.sort_values('mean', ascending=False)
display(summary_df)

In [ ]:
# Probabilités de majorité
print("\nProbabilité de majorité absolue (82 sièges) :")
print("="*50)
for coalition, prob in mc_result.majority_probabilities.items():
    print(f"  {coalition}: {prob*100:.1f}%")

In [ ]:
# Histogramme de la distribution pour la liste en tête
top_liste = summary_df.index[0]
dist = mc_result.seats_distributions.get(top_liste, np.array([]))
ci = mc_result.seats_ci(top_liste)

if len(dist) > 0:
    fig = histogram_mc(dist, top_liste, ci)
    fig.show()
else:
    print(f"Pas de données pour {top_liste}")

## 5. Comparaison des distributions

In [ ]:
# Box plots des distributions de sièges
box_data = []
for liste, dist in mc_result.seats_distributions.items():
    for val in dist:
        box_data.append({'Liste': liste, 'Sièges': val})

box_df = pd.DataFrame(box_data)

fig = px.box(
    box_df,
    x='Liste',
    y='Sièges',
    color='Liste',
    color_discrete_map={k: POLITICAL_FAMILIES.get(k, POLITICAL_FAMILIES['DIV']).color 
                        for k in box_df['Liste'].unique()},
    title=f'Distribution des sièges — Monte Carlo (N={N})',
)

# Ligne de majorité
fig.add_hline(y=MAYOR_ABSOLUTE_MAJORITY, line_dash='dash', line_color='red',
              annotation_text=f'Majorité ({MAYOR_ABSOLUTE_MAJORITY})')

fig.update_layout(template='plotly_white', showlegend=False)
fig.show()

## 6. Convergence Monte Carlo

Vérification que les résultats sont stables.

In [ ]:
# Convergence de la moyenne des sièges
if top_liste in mc_result.seats_distributions:
    dist = mc_result.seats_distributions[top_liste]
    running_mean = np.cumsum(dist) / np.arange(1, len(dist) + 1)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=running_mean, mode='lines', name='Moyenne cumulée'))
    fig.add_hline(y=np.mean(dist), line_dash='dash', line_color='red',
                  annotation_text=f'Moyenne finale: {np.mean(dist):.1f}')
    
    fig.update_layout(
        title=f'Convergence Monte Carlo — {top_liste}',
        xaxis_title='Itération',
        yaxis_title='Moyenne sièges',
        template='plotly_white',
    )
    fig.show()

## 7. Conclusions

### Observations clés :

1. **Incertitude élevée** : les intervalles de confiance sont larges (±10-15 sièges)
2. **Majorité incertaine** : aucune coalition n'a de probabilité de majorité > 50%
3. **Sensibilité à la participation** : une faible participation favorise...

### Limites :

- Modèle de redressement calibré sur peu de données
- Transferts de voix T2 très incertains
- Dynamique de campagne non modélisée

### Recommandations :

1. Utiliser N=10 000 pour des résultats robustes
2. Explorer plusieurs scénarios de transferts T2
3. Mettre à jour les scores au fil des sondages